In [6]:
import helper_functions as hf
from helper_functions.classes.player import Player
from helper_functions.setup.email_handling import get_email_address
from IPython.display import Markdown, HTML
import win32clipboard
import markdown
import pandas as pd

def copy_to_clipboard(text: str):
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardText(text, win32clipboard.CF_UNICODETEXT)
    win32clipboard.CloseClipboard()


email_base_text = hf.DATAPATH.joinpath("helper_texts/email_blank_text.md").read_text(encoding="utf-8")
hidden_df = hf.FpathRegistry.get_hidden_responses().set_index("nickname")
df = hf.get_players().set_index("nickname", drop=False).fillna("")
df.head()

df = df.join(hidden_df[[col for col in hidden_df.columns if not col in df.columns]], how="left")



In [10]:
def get_email_text_for_player(player: pd.Series) -> tuple[str, str]:
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    team_name = player["Team"]
    cloth_color = {"Team A": "dark", "Team B": "white", "Team C": "colorful"}
    email = get_email_address(player["email"])
    
    player_obj = Player.from_series(player, list(hf.ALL_MATCHES))
    schedule = player_obj.get_schedule()
    return email, email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, team_name=team_name, cloth_color=cloth_color)

player = df.loc["Thankful Kakapo"]
address, text = get_email_text_for_player(player)
text = markdown.markdown(text.replace("\\", "<br>"))
copy_to_clipboard(text)


In [74]:
df.columns


Index(['nickname', 'institute', 'is_postdoc', 'avail_monday', 'avail_tuesday',
       'avail_thursday', 'avail_friday', 'wants_basketball', 'basketball',
       'wants_running_sprints', 'running_sprints', 'wants_volleyball',
       'volleyball', 'wants_chess', 'chess', 'wants_football', 'football',
       'wants_tennis', 'tennis', 'wants_capture_the_flag', 'capture_the_flag',
       'wants_spikeball', 'spikeball', 'wants_beer_pong', 'beer_pong',
       'wants_foosball', 'foosball', 'wants_ping_pong', 'ping_pong',
       'num_sports', 'num_sports_not_avail', 'late_entry', 'Team',
       'subteam_basketball', 'subteam_running_sprints', 'subteam_volleyball',
       'subteam_chess', 'subteam_football', 'subteam_tennis',
       'subteam_capture_the_flag', 'subteam_spikeball', 'subteam_beer_pong',
       'subteam_foosball', 'subteam_ping_pong', 'num_sports_attending',
       'attendance_ratio', 'response_timestamp', 'name', 'phd_or_postdoc',
       'time_available', 'events_interested_in', '

In [ ]:


player = Player.from_series(df.loc["Animated Yak"], list(hf.ALL_MATCHES))
player.get_schedule()

sched = player.get_schedule()

text_to_copy = "<b>Hello, World!🏀</b>"
md_text = markdown.markdown(sched.replace("\\", "<br>"))
copy_to_clipboard(md_text)


   ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/105.4 kB ? eta -:--:--
   ---------------------------------------- 105.4/105.4 kB 1.5 MB/s eta 0:00:00
